In [1]:
#importando pacotes úteis
import pandas as pd
import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn import neighbors

In [2]:
dt_2apli_ruido_nominal = pd.read_csv('2apli_25ruido_25incerteza.csv');
dt_2apli_ruido_nominal

,App1 - SLLC,App1 - DRAM,App1 - NET,App2 - SLLC,App2 - DRAM,App2 - NET,Nivel de interferencia
0,Muito alto,Muito baixo,Muito baixo,Muito alto,Muito baixo,Muito baixo,Alto
1,Muito alto,Muito baixo,Muito baixo,Medio,Muito baixo,Muito baixo,Baixo
2,Muito alto,Muito baixo,Muito baixo,Muito baixo,Muito baixo,Muito baixo,Muito baixo
3,Muito alto,Muito baixo,Muito baixo,Baixo,Muito alto,Muito baixo,Baixo
4,Muito alto,Muito baixo,Muito baixo,Muito baixo,Medio,Muito baixo,Muito baixo
...,...,...,...,...,...,...,...
221,Muito baixo,Muito baixo,Muito baixo,Muito baixo,Muito baixo,Muito baixo,Muito baixo
222,Muito baixo,Muito baixo,Muito baixo,Muito baixo,Muito baixo,Muito baixo,Muito baixo
223,Muito baixo,Muito baixo,Muito baixo,Muito baixo,Muito baixo,Medio,Muito baixo
224,Muito baixo,Muito baixo,Muito baixo,Muito baixo,Muito baixo,Medio,Muito baixo


In [3]:
dt_2apli_ruido_nominal['Nivel de interferencia'].value_counts()

Muito baixo    120
Baixo           68
Medio           28
Alto             6
Muito alto       4
Name: Nivel de interferencia, dtype: int64

In [4]:
erro_majoritario = dt_2apli_ruido_nominal['Nivel de interferencia'].value_counts().max()/dt_2apli_ruido_nominal['Nivel de interferencia'].count()
print("erro_majoritario = ", erro_majoritario) 

erro_majoritario =  0.5309734513274337


In [5]:
dict_cat = {
    'Muito baixo':0,
    'Baixo':1,
    'Medio':2,
    'Alto':3,
    'Muito alto': 4
}

In [6]:
#O K-NN só funciona com valores numéricos
dt_2apli_ruido = pd.DataFrame(columns=['App1 - SLLC', 'App1 - DRAM', 'App1 - NET', 'App2 - SLLC', 'App2 - DRAM', 'App2 - NET'])
dt_2apli_ruido['App1 - SLLC'] = dt_2apli_ruido_nominal['App1 - SLLC'].map(dict_cat)
dt_2apli_ruido['App1 - DRAM'] = dt_2apli_ruido_nominal['App1 - DRAM'].map(dict_cat)
dt_2apli_ruido['App1 - NET'] = dt_2apli_ruido_nominal['App1 - NET'].map(dict_cat)
dt_2apli_ruido['App2 - SLLC'] = dt_2apli_ruido_nominal['App2 - SLLC'].map(dict_cat)
dt_2apli_ruido['App2 - DRAM'] = dt_2apli_ruido_nominal['App2 - DRAM'].map(dict_cat)
dt_2apli_ruido['App2 - NET'] = dt_2apli_ruido_nominal['App2 - NET'].map(dict_cat)
dt_category = dt_2apli_ruido_nominal['Nivel de interferencia'].map(dict_cat)
dt_target = np.array(dt_category)
dt_data = np.array(dt_2apli_ruido)
print('dt_target = ', dt_target)
print('dt_data = ', dt_data)

dt_target =  [3 1 0 1 0 2 4 2 1 1 0 0 0 1 2 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 0 0 0 0 0 3 2
 1 1 4 2 1 2 1 1 2 1 0 1 0 2 2 1 1 2 1 2 1 0 0 1 0 1 1 0 0 0 1 2 1 0 1 1 3
 2 1 1 3 4 1 0 1 0 2 4 2 1 2 1 2 3 2 0 2 0 2 1 1 0 0 0 1 2 2 0 1 0 1 1 0 0
 0 1 0 0 0 0 0 0 1 1 1 1 0 1 0 0 0 0 1 0 0 1 0 1 2 1 1 1 1 1 1 0 1 0 0 0 0
 0 1 1 0 0 1 0 1 0 1 2 1 0 1 0 2 2 1 1 2 1 3 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]
dt_data =  [[4 0 0 4 0 0]
 [4 0 0 2 0 0]
 [4 0 0 0 0 0]
 ...
 [0 0 0 0 0 2]
 [0 0 0 0 0 2]
 [0 0 0 0 0 0]]


In [7]:
loo = LeaveOneOut()
loo.get_n_splits(dt_data)
count_rights = [0] * 10
count_wrongs = [0] * 10
for train_index, test_index in loo.split(dt_data):
    #print("Test_index = ", test_index)
    dt_data_train = dt_data[train_index]
    dt_target_train = dt_target[train_index]
    dt_data_test = dt_data[test_index]
    dt_target_test = dt_target[test_index]
    for n_neighbors in range(1, 10, 1):
        clf = neighbors.KNeighborsClassifier(n_neighbors, weights='uniform')
        clf.fit(dt_data_train, dt_target_train)
        
        dt_z_test = clf.predict(dt_data_test)
        
        #print("n_neighbors = ", n_neighbors)
        if(dt_z_test == dt_target_test):
            #print("acertou")
            count_rights[n_neighbors] = count_rights[n_neighbors] + 1
        else:
            #print("errou")
            count_wrongs[n_neighbors] = count_wrongs[n_neighbors] + 1

In [8]:
print("Acertos = ", count_rights)
print("Erros = ", count_wrongs)

Acertos =  [0, 144, 148, 145, 150, 152, 153, 146, 153, 146]
Erros =  [0, 82, 78, 81, 76, 74, 73, 80, 73, 80]


In [9]:
n_elements = [0]
precision = [0]
for i in range(1, 10, 1):
    n_elements.append(count_rights[i] + count_wrongs[i])
    precision.append(count_rights[i]/n_elements[i])
print("n_elements = ", n_elements[1:])
print("precision = ", precision[1:])
#precision = np.true_divide(count_rights,215)
#precision

n_elements =  [226, 226, 226, 226, 226, 226, 226, 226, 226]
precision =  [0.6371681415929203, 0.6548672566371682, 0.6415929203539823, 0.6637168141592921, 0.672566371681416, 0.6769911504424779, 0.6460176991150443, 0.6769911504424779, 0.6460176991150443]
